In [2]:
from datetime import date

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygris
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

from epymorph.data import geo_library, ipm_library, mm_library
from epymorph.initializer import bird_movement_initializer
from epymorph.simulation import Simulation

In [3]:
geo = geo_library['maricopa_cbg_2019']()

source_node = '040130927211'
birds = np.zeros(geo.nodes)
birds[np.where(geo['geoid'] == source_node)] = 50000
geo.values['bird_population'] = birds

In [6]:
sim = Simulation(
    geo=geo,
    ipm_builder=ipm_library['no'](),
    mvm_builder=mm_library['local_birds'](),
)

out = sim.run(
    param={
        'shape': 3,
        'scale':3
    },
    start_date=date(2023, 1, 1),
    duration_days=10,
    initializer=bird_movement_initializer,
)

In [7]:
df = pd.merge(
    left=pygris.block_groups(
        state="Arizona",
        county="Maricopa",
        year=2019,
    ),
    right=pd.DataFrame({
        'GEOID': geo['geoid'],
        'visitors': out.prevalence[0, :, 0],
    }),
    on='GEOID',
)

Using FIPS code '04' for input 'Arizona'
Using FIPS code '013' for input 'Maricopa'
0           0
1           0
2          38
3       45000
4           6
        ...  
2489        0
2490        0
2491        0
2492        0
2493        0
Name: visitors, Length: 2494, dtype: int64


In [10]:
print(df)

     STATEFP COUNTYFP TRACTCE BLKGRPCE         GEOID       NAMELSAD  MTFCC  \
0         04      013  421702        3  040134217023  Block Group 3  G5030   
1         04      013  040531        2  040130405312  Block Group 2  G5030   
2         04      013  092312        1  040130923121  Block Group 1  G5030   
3         04      013  092721        1  040130927211  Block Group 1  G5030   
4         04      013  093104        1  040130931041  Block Group 1  G5030   
...      ...      ...     ...      ...           ...            ...    ...   
2489      04      013  811900        2  040138119002  Block Group 2  G5030   
2490      04      013  812000        1  040138120001  Block Group 1  G5030   
2491      04      013  812000        2  040138120002  Block Group 2  G5030   
2492      04      013  523002        3  040135230023  Block Group 3  G5030   
2493      04      013  523102        3  040135231023  Block Group 3  G5030   

     FUNCSTAT    ALAND  AWATER     INTPTLAT      INTPTLON  \
0 

In [1]:
fig, ax = plt.subplots(figsize=(10, 8))
norm = Normalize(vmin=0, vmax=20)
sm = ScalarMappable(cmap='Reds', norm=norm)
df.plot(ax=ax, color=sm.to_rgba(df['visitors'].to_numpy()))
cbar = plt.colorbar(sm, ax=ax, orientation='vertical')
cbar.set_label('No of birds moved')
plt.title('Bird Movement')
plt.show()

NameError: name 'plt' is not defined